<a href="https://colab.research.google.com/github/AbdulMoiz-MMY/Agentic_AI_Learning/blob/main/Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing langchain_google_genai

In [ ]:
!pip install -q -U langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.4 MB/s eta 0:00:00


# Importing Neccessary Libraries

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

# Setting Up API Key

In [ ]:
GOOGLE_APi_KEY : str = userdata.get("GOOGLE_API_KEY")

# Initializing Language Model

In [ ]:
llm:ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(api_key=GOOGLE_APi_KEY,model="gemini-2.0-flash-exp")

# Invoking the Model

In [ ]:
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0d85500a-0bbc-4fdc-b155-5b959bec472d-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

# Defining Custom Tools

In [ ]:
from langchain_core.tools import tool
import random

@tool
def emoji_translator(text: str) -> str:
    """Translates words,emotions like 'happy', 'sad', 'dog', 'cat' into emojis from user sentences."""
    emoji_dict = {"happy": "😊", "sad": "😢", "dog": "🐶", "cat": "🐱"}
    for word, emoji in emoji_dict.items():
        text = text.replace(word, emoji)
    return text

@tool
def is_palindrome(text: str) -> bool:
    """Checks if the given text is a palindrome, ignoring case, spaces, commas, and periods."""
    text = text.lower().replace(" ", "").replace(",", "").replace(".", "")
    return text == text[::-1]

@tool
def random_fact() -> str:
    """Returns a random fact from a predefined list."""
    facts = ["Honey never spoils.", "Octopuses have three hearts."]
    return random.choice(facts)

tools = [emoji_translator,is_palindrome,random_fact]

# Binding Tools To the Language Model

In [ ]:
llm_with_tools = llm.bind_tools(tools)

# Invoking the model with Tools

In [ ]:
llm_with_tools.invoke("Happy")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'emoji_translator', 'arguments': '{"text": "Happy"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-d6878b73-276b-476b-bdaa-73ab8ec82739-0', tool_calls=[{'name': 'emoji_translator', 'args': {'text': 'Happy'}, 'id': 'd81c6461-78b2-4b99-85a4-807d3176edb3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 5, 'total_tokens': 144, 'input_token_details': {'cache_read': 0}})

# Query To Use Tool (Part_1)

In [ ]:
from langchain_core.messages import HumanMessage,SystemMessage
system_message = SystemMessage(content="You are a helpful assistant that uses tools to answer user queries. Always use the appropriate tool when asked.")
query = "Call the emoji_translator tool to translate the following sentence into emojis: I have a happy dog."
messages = [system_message,HumanMessage(content=query)]

In [ ]:
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [ ]:
ai_msg.tool_calls

[{'name': 'emoji_translator',
  'args': {'text': 'I have a happy dog.'},
  'id': 'd7581b3c-3e67-416c-88bf-c131f7272c0a',
  'type': 'tool_call'}]

In [ ]:
print("Gemini Response:", ai_msg)

Gemini Response: content='' additional_kwargs={'function_call': {'name': 'emoji_translator', 'arguments': '{"text": "I have a happy dog."}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-d7cbc771-2eeb-44e7-9b41-2d27225e3c06-0' tool_calls=[{'name': 'emoji_translator', 'args': {'text': 'I have a happy dog.'}, 'id': 'd7581b3c-3e67-416c-88bf-c131f7272c0a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 179, 'output_tokens': 10, 'total_tokens': 189, 'input_token_details': {'cache_read': 0}}


In [ ]:
for tool_call in ai_msg.tool_calls:
  selected_tool = {
      "emoji_translator": emoji_translator,
      "is_palindrome": is_palindrome,
      "random_fact": random_fact,
  }[tool_call["name"].lower()]
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)

In [ ]:
from IPython.display import Markdown
response = llm_with_tools.invoke(messages)
display(Markdown(response.content))

I have a 😊 🐶.

# Query To Use Tool(Part_2)

In [ ]:
from langchain_core.messages import HumanMessage

query:str = "Fact"
messages = [HumanMessage(query)]
display(messages)

[HumanMessage(content='Fact', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
display(messages)

[HumanMessage(content='Fact', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'random_fact', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-186df784-a588-4b28-b906-4b379afe016a-0', tool_calls=[{'name': 'random_fact', 'args': {}, 'id': 'ab179736-1e6f-49b4-b294-288aa4a76c9a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 3, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}})]

In [ ]:
ai_msg.tool_calls

[{'name': 'random_fact',
  'args': {},
  'id': 'ab179736-1e6f-49b4-b294-288aa4a76c9a',
  'type': 'tool_call'}]

In [ ]:
for tool_call in ai_msg.tool_calls:
  selected_tool={
      "emoji_translator":emoji_translator,
      "is_palindrome":is_palindrome,
      "random_fact":random_fact,
  }[tool_call["name"].lower()]
  tool_msg = selected_tool.invoke(tool_call)
  display(tool_msg)
  messages.append(tool_msg)

ToolMessage(content='Octopuses have three hearts.', name='random_fact', tool_call_id='ab179736-1e6f-49b4-b294-288aa4a76c9a')

In [ ]:
messages

[HumanMessage(content='Fact', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'random_fact', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-186df784-a588-4b28-b906-4b379afe016a-0', tool_calls=[{'name': 'random_fact', 'args': {}, 'id': 'ab179736-1e6f-49b4-b294-288aa4a76c9a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 3, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='Octopuses hav

In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
from IPython.display import Markdown
display(Markdown(response.content))

Octopuses have three hearts.